In [4]:
import pandas as pd

In [2]:
# Subir el archivo de navegacion a Colab
from google.colab import files
uploaded = files.upload()

Saving navegacion.csv to navegacion.csv


In [ ]:
# crear el dataframe a partir del fichero de navegacion
dfNavegacion = pd.read_csv("navegacion.csv", sep=";")
dfNavegacion.head(10)

EDA (Analisis Exploratorio de Datos)

In [6]:
# comprobar cuantos gclid repetidos tengo
print(dfNavegacion['gclid'].value_counts().head(50))

EAIaIQobChMIwPu5t4qs3AIVAQAAAB0BAAAAEAAYACAAEgJVzfD_BwE                                         124
0                                                                                                35
CjwKCAjwjbCDBhAwEiwAiudBy3jlJeW4vAzqieU59RXrhxtMvbHXgK5UdK0KHtUCtcp6GWd38D0u6BoC76EQAvD_BwE      10
Cj0KCQjw-NaJBhDsARIsAAja6dMELUkx0xI6udIqEYLUIQSpbP0FQtNk2aVxcUiMQUjb058AhqTsvhcaAho4EALw_wcB      5
Cj0KCQjw-NaJBhDsARIsAAja6dOww1IgSWThNMRCIYMOqCLia0Udue7wdypptdIMPRlUBCr6gTi-_C4aAgdIEALw_wcB      5
Cj0KCQjw-NaJBhDsARIsAAja6dOmcKjgn77p5hHT_5b1XIyg6fHlpWRs8rdXlMlnlzQsJezWS4JyGQYaAv7DEALw_wcB      4
Cj0KCQjw-NaJBhDsARIsAAja6dPF-dZH3B6kU3Lp_q8fn9f8bl4tz8uOoZbmJJcL6EkgWPG6mvqGRZ0aAoWoEALw_wcB      4
Cj0KCQjw-NaJBhDsARIsAAja6dOMdiySET1yjbRJgb3qi8RdOv3ETY-RRl1FLazS7VULGSOxx18mEVUaAhcea?Lw_wcB      4
Cj0KCQjw-NaJBhDsARIsAAja6dPLhRHDqi-qYz4dcE9XFVvOcq4o0eiCW5ss8t3T_h7u9VB4NXAS-FkaAhKzEALw_wcB      4
Cj0KCQjw-NaJBhDsARIsAAja6dPDldGfiETrgOJl8913cSBMnZ30bWT4i0PvY9XYWSF39CF__gMmYqcaAhWsEALw_wcB      4


In [ ]:
# Hemos visto que el gclid=0 pertenece a diferentes usuarios, por lo
# que no es un dato coherente y lo elimino
gclid0 = dfNavegacion[dfNavegacion['gclid'] == '0'].index
print(gclid0)
dfNavegacion = dfNavegacion.drop(gclid0)

In [ ]:
# tambien vamos a borrar el gclid que tenia 124 ocurrencias
gclid124 = dfNavegacion[dfNavegacion['gclid'] == 'EAIaIQobChMIwPu5t4qs3AIVAQAAAB0BAAAAEAAYACAAEgJVzfD_BwE'].index
print(gclid124)
dfNavegacion = dfNavegacion.drop(gclid124)

Empezamos a procesar la url que tenemos mucha informacion util


In [11]:
# Me quedo solo con las url no nulas
serieUrlLanding = dfNavegacion['url_landing'][dfNavegacion['url_landing'].notnull()]

# vamos hacer split con la url y generar una columna con cada parametro
dfNavegacion[['url_base','idUser','uuid_url','camp','adg','device','sl','adv','rec','vacia']] = serieUrlLanding.str.split(pat='&',n=10,expand=True)

In [12]:
# Elimino las colunas que no voy a necesitar
dfNavegacion = dfNavegacion.drop(['idUser','uuid_url','rec', 'vacia'], axis=1)

In [14]:
dfNavegacion['url_base'].head(20)
# url/modelo?parametros
# la parte de la url que me intersa esta por delante de la interrogacion
dfNavegacion['url_base'] = dfNavegacion['url_base'].apply(lambda url: str(url).split('?')[0])
dfNavegacion['url_base'].head(20)

0                  https://www.metropolis.com/es/
1     https://www.metropolis.com/es/ixs-electrico
2              https://www.metropolis.com/es/tria
3              https://www.metropolis.com/es/tria
4                  https://www.metropolis.com/es/
5               https://www.metropolis.com/es/cea
8           https://www.metropolis.com/es/clin200
9                  https://www.metropolis.com/es/
10                 https://www.metropolis.com/es/
11          https://www.metropolis.com/es/clin400
12          https://www.metropolis.com/es/clin200
13             https://www.metropolis.com/es/tria
14              https://www.metropolis.com/es/cea
15                 https://www.metropolis.com/es/
16                 https://www.metropolis.com/es/
17              https://www.metropolis.com/es/cea
18                 https://www.metropolis.com/es/
19     https://www.metropolis.com/es/tria-hibrido
20             https://www.metropolis.com/es/tria
21            https://www.metropolis.com/es/dep30


In [15]:
# Obtener el modelo de coche y crear una columna
def obtenerModelo(url):
    posicion = url.rfind('/')
    modeloObtenido = url[posicion+1:]
    if modeloObtenido == '' or modeloObtenido == 'renting':
        modeloObtenido = 'home'
    return modeloObtenido

dfNavegacion.insert(loc=7, column='modelo', value=dfNavegacion['url_base'].apply(obtenerModelo))
print(dfNavegacion['modelo'].head(50))

0                  home
1         ixs-electrico
2                  tria
3                  tria
4                  home
5                   cea
8               clin200
9                  home
10                 home
11              clin400
12              clin200
13                 tria
14                  cea
15                 home
16                 home
17                  cea
18                 home
19         tria-hibrido
20                 tria
21                dep30
22                 home
23                 tria
24                dep30
25                dep30
26        cea-electrico
27              clin400
28                 home
29                 home
30                 tria
31                 tria
32                 home
33         tria-hibrido
35              clin200
36                 tria
37                 home
38                 home
39                dep30
40               dep30F
42                 home
43                 home
44                 home
45              

Crear un orden de usuarios.
Como tenemos usuarios repetidos me daria un problema al hacer el merge con el fichero de conversiones.

In [16]:
# Crear una columna llamada orden que almacene el orden de los id_user
dfNavegacion = dfNavegacion.sort_values(['id_user','ts'])
dfNavegacion.insert(loc=1,column='orden', value=dfNavegacion.groupby('id_user')['ts'].cumcount())

In [17]:
# Filtrar por orden=0
dfNavegacion = dfNavegacion[dfNavegacion['orden'] == 0] 

Procesamos el fichero de conversiones

In [18]:
# subir el fichero de conversiones a Colab
from google.colab import files
uploaded = files.upload()

Saving conversiones.csv to conversiones.csv


In [ ]:
# Crear del dataframe de conversiones
dfConversiones = pd.read_csv("conversiones.csv", sep=";")
dfConversiones.head(20)

In [20]:
# Vamos a hacer el merge con el modelo outer para no perder nada de informacion
dfFinal = dfNavegacion.merge(dfConversiones, how='outer', on=['id_user'])

In [21]:
# Crear una nueva columna llamada conversiones con el valor:
#     1 si es conversion, 0 si no lo es
def esConversion(dato):
  lista = ['CALL','FORM']
  if dato in lista:
    return 1
  else:
    return 0

dfFinal.insert(loc=1, column='conversiones', value=dfFinal['lead_type'].apply(esConversion))

**************** PREGUNTAS ****************


In [ ]:
# Pregunta 1: Cuántas visitas recibe en el dia el cliente?
dfFinal['ts'].head(50)

# crear una columna con el dia obtenido de la columna ts
# utilizo el split cortando por el espacio
dfFinal.insert(loc=1, column='dia', value=dfFinal['ts'].apply(lambda ts: str(ts).split(' ')[0]))
dfFinal['dia'].head(50)


In [25]:
# contamos visitas por dia
dfFinal.groupby('dia')['dia'].count()

dia
2021-09-06    5640
2021-09-07       4
nan             13
Name: dia, dtype: int64

In [29]:
# Cuantas convierten y cuantas no? En porcentaje

# Filtrar por 2021-09-06 y las cuento
visitas_dia_6_septiembre = dfFinal[dfFinal['dia'] == '2021-09-06']
num_visitas_dia = visitas_dia_6_septiembre['dia'].count()
num_visitas_dia

# contar las visitas con conversion
visitas_convierten = visitas_dia_6_septiembre[visitas_dia_6_septiembre['conversiones']==1]['ts'].count()

# contar las visitas sin conversion
visitas_no_convierten = visitas_dia_6_septiembre[visitas_dia_6_septiembre['conversiones']==0]['ts'].count()

# Calculo porcentajes
porcentaje_visitas_convierten = round( (visitas_convierten / num_visitas_dia) * 100, 2)
porcentaje_visitas_no_convierten = round( (visitas_no_convierten / num_visitas_dia) * 100, 2)
print("Convierten:", porcentaje_visitas_convierten)
print("No convierten:", porcentaje_visitas_no_convierten)


Convierten: 0.27
No convierten: 99.73


In [30]:
# Pregunta 2. Por tipo de conversion (CALL o FORM) cuantas hay de cada una de ellas
print(dfFinal.groupby('lead_type')['lead_type'].count())

lead_type
CALL    19
FORM     9
Name: lead_type, dtype: int64


In [31]:
# Pregunta 3. Calcular el porcentaje de usuarios recurrentes sobre el total de usuarios
# Contar cuantos usuarios tenemos
total_usuarios = dfFinal['id_user'].count()

# Contar los usuarios recurrentes
usuarios_recurrentes = dfFinal[dfFinal.user_recurrent & dfFinal.user_recurrent.notnull()]['ts'].count()

# calculamos el porcentaje
porcentaje = round((usuarios_recurrentes / total_usuarios) * 100 , 2)
print("Usuarios recurrentes: ", porcentaje)


Usuarios recurrentes:  18.7


In [33]:
# Pregunta 4. Coche mas visitado
modelos_mas_visitados = dfFinal['modelo'].value_counts()

# Eliminar home
modelos_mas_visitados =  modelos_mas_visitados.drop(labels=['home'])

# Quedarnos solo con el primero
modelos_mas_visitados = modelos_mas_visitados.head(1)
modelos_mas_visitados

tria    1167
Name: modelo, dtype: int64

In [36]:
# Es el coche que mas convierte?
# Filtro del dataset para quedarme solo los registros que han convertido
# los agrupo por modelo
print(dfFinal[dfFinal['conversiones'] == 1].groupby('modelo')['conversiones'].count())

modelo
cea-electrico    1
clin200          2
clin400          3
dep30            1
home             5
ixs-electrico    1
tria             2
Name: conversiones, dtype: int64


In [37]:
# Exportal el dataframe final a Excel, para poder elaborar informes, graficos, histogramas, ...etc
dfFinal.to_excel("dataframe_final.xlsx", index=False)